# dN - dStop - dS on usher SARS-Cov-2 tree
This notebook is based loosely off Katie's original dN-dStop-dS notebook on a nextstrain ncov tree. Here I'm applying the same idea to the Usher SARS-CoV-2 tree. 

One key point that took me some time to get: There is no time in this calculation! It is just the number of synonymous or nonsynonymous or stop mutations over the expected number of such sites based on the genome sequence and a substitution matrix. That's why we divide by dS because these values aren't really interpretable without dS.

In [1]:
import os
from Bio import SeqIO
import pandas as pd
import numpy as np

In [2]:
os.chdir('/Users/cwagner2/Work/projects/covid/long-deletions/')

## 1. Generate expected number of sites
This is all Kathryn Kistler's OG code

In [3]:
location_by_gene = {}

#with open("reference_seq_S1.gb") as reference_handle:
with open("sars2_wS1_ref.gb") as reference_handle:
    for record in SeqIO.parse(reference_handle, "genbank"):
        wuhan_seq = record.seq
        for feature in record.features:
            if feature.type == 'CDS':
                gene = feature.qualifiers['gene'][0]
                location = feature.location
                location_by_gene[gene] = location



In [4]:
location_by_gene

{'ORF1a': SimpleLocation(ExactPosition(265), ExactPosition(13468), strand=1),
 'ORF1b': SimpleLocation(ExactPosition(13467), ExactPosition(21555), strand=1),
 'S': SimpleLocation(ExactPosition(21562), ExactPosition(25384), strand=1),
 'S1': SimpleLocation(ExactPosition(21598), ExactPosition(23617), strand=1),
 'ORF3a': SimpleLocation(ExactPosition(25392), ExactPosition(26220), strand=1),
 'E': SimpleLocation(ExactPosition(26244), ExactPosition(26472), strand=1),
 'M': SimpleLocation(ExactPosition(26522), ExactPosition(27191), strand=1),
 'ORF6': SimpleLocation(ExactPosition(27201), ExactPosition(27387), strand=1),
 'ORF7a': SimpleLocation(ExactPosition(27393), ExactPosition(27759), strand=1),
 'ORF7b': SimpleLocation(ExactPosition(27755), ExactPosition(27887), strand=1),
 'ORF8': SimpleLocation(ExactPosition(27893), ExactPosition(28259), strand=1),
 'N': SimpleLocation(ExactPosition(28273), ExactPosition(29533), strand=1),
 'ORF9b': SimpleLocation(ExactPosition(28283), ExactPosition(28

In [5]:
all_genes = list(location_by_gene.keys())

In [6]:
#extract the sequence of each gene, and find it's constituent codon sequences
codons_by_gene = {}

for g, l in location_by_gene.items():
    gene_sequence = l.extract(wuhan_seq)
    #split into codons
    gene_codons = [gene_sequence[i:i+3] for i in range(0, len(gene_sequence), 3)]
    codons_by_gene[g] = gene_codons
    
#because indexing is 0-based, but codons count from 1, need to use codons_by_gene[gene][codon-1]

In [7]:
#make mapper of nt position to codon position (within gene)
#codon_index_by_pos = {}
#for g, l in location_by_gene.items():
    #map to S, not S1. And map to N, not 9b
 #   if g not in ['S1', 'ORF9b']:
        #start at 1!
  #      positions_grouped_into_codons = [[i+1,i+2,i+3] for i in range(l.start, l.end, 3)]
   #     for i in range(len(positions_grouped_into_codons)):
    #        for j in range(len(positions_grouped_into_codons[i])):
     #           pos = positions_grouped_into_codons[i][j]
      #          codon_index_by_pos[pos] = {'gene':g, 'codon': i+1, 'codon_pos': j}
            
#for all positions not in a gene, specify 'noncoding'
#for x in range(len(wuhan_seq)+1):
 #   if x not in codon_index_by_pos.keys():
  #      codon_index_by_pos[x] = {'gene':'Noncoding'}

In [8]:
#make mapper of nt position to codon position within orf9b
#codon_index_by_pos_orf9b = {}
#for g, l in location_by_gene.items():
 #   if g == 'ORF9b':
  #      #start at 1!
   #     positions_grouped_into_codons = [[i+1,i+2,i+3] for i in range(l.start, l.end, 3)]
    #    for i in range(len(positions_grouped_into_codons)):
     #       for j in range(len(positions_grouped_into_codons[i])):
      #          pos = positions_grouped_into_codons[i][j]
       #         codon_index_by_pos_orf9b[pos] = {'gene':g, 'codon': i+1, 'codon_pos': j}

In [9]:
#substitution matrix from https://www.sciencedirect.com/science/article/pii/S1567134821004287
#format is {from_A:{to_T:x, to_C:x, to_G:x}, from_T:{to_A:x, ...}, ...}

sub_matrix = {'A': {'T': 0.0383, 'C': 0.0219, 'G': 0.0747},
          'T': {'A': 0.0356, 'C': 0.2085, 'G': 0.0234},
          'C': {'A': 0.0356, 'T': 0.3648, 'G': 0.0234},
          'G': {'A': 0.1138, 'T': 0.0383, 'C': 0.0219}}

In [10]:
all_nts = ['A', 'T', 'C', 'G']

In [11]:
#introduce every possible mutation * probability that that mutation occurs (given A->T, oR A->G, etc), 
#and find whether it is synonymous or not
#total the number of synonymous and nonsynonymous sites per gene

syn_denominators ={}
nonsyn_denominators ={}
stop_denominators = {}

for gene, codons in codons_by_gene.items():
    nonsyn_sites_per_gene = 0
    syn_sites_per_gene = 0
    stop_sites_per_gene = 0
    
    #walk through codons
    for codon in codons:
        original_aa = codon.translate()
        
        #for each position in the codon
        for i in range(len(codon)):
            #wuhan nt
            nt = codon[i]
            #introduce each other mutation
            for mut in [x for x in all_nts if x!=nt]:
                mut_codon = codon[:i]+mut+codon[i+1:]
                mut_aa = mut_codon.translate()
                #find whether nonsynonymous
                if mut_aa != original_aa:
                    #find whether this is a stop codon
                    if mut_aa =='*':
                        #get the probability of this mutation
                        #add to the total number of stop sites
                        stop_sites_per_gene+=sub_matrix[nt][mut]
                    else:
                        #or the total number of nonsyn sites
                        nonsyn_sites_per_gene+=sub_matrix[nt][mut]
                #or synonymous
                elif mut_aa == original_aa:
                    syn_sites_per_gene+=sub_matrix[nt][mut]
                    
    syn_denominators[gene] = syn_sites_per_gene
    nonsyn_denominators[gene] = nonsyn_sites_per_gene
    stop_denominators[gene] = stop_sites_per_gene

In [12]:
syn_denominators

{'ORF1a': 965.313399999992,
 'ORF1b': 589.2138000000007,
 'S': 285.77269999999635,
 'S1': 153.71709999999982,
 'ORF3a': 64.80060000000032,
 'E': 19.474200000000007,
 'M': 54.699800000000174,
 'ORF6': 12.598300000000005,
 'ORF7a': 27.299699999999966,
 'ORF7b': 11.153400000000003,
 'ORF8': 26.9515,
 'N': 95.4907000000009,
 'ORF9b': 22.801499999999994}

In [13]:
nonsyn_denominators

{'ORF1a': 2036.4235999996595,
 'ORF1b': 1245.31339999995,
 'S': 595.1919000000037,
 'S1': 316.35069999999456,
 'ORF3a': 131.76400000000203,
 'E': 37.00099999999997,
 'M': 105.84390000000141,
 'ORF6': 27.751999999999896,
 'ORF7a': 59.49950000000032,
 'ORF7b': 21.03959999999997,
 'ORF8': 57.00870000000031,
 'N': 191.62639999999817,
 'ORF9b': 46.86320000000012}

## 2. Make mutations df

In [169]:
with open('usher/trimmed/usher_translations.tsv','r') as f:
    muts = pd.read_csv(f,sep='\t')

In [170]:
muts.iloc[0:1000,:].to_csv('usher/trimmed/test_translations.tsv',sep='\t',index=False)

In [15]:
## Split up nodes with multiple mutations into single mutations
muts['aa_mutations'] = muts.aa_mutations.str.split(';')
muts['nt_mutations'] = muts.nt_mutations.str.split(';')
muts['codon_changes'] = muts.codon_changes.str.split(';')
all_muts = muts.explode(['aa_mutations','nt_mutations','codon_changes'],ignore_index=True)

In [16]:
## Label with gene
all_muts[['gene','aa_mutation']] = all_muts['aa_mutations'].str.split(':',expand=True)
all_muts.drop(columns=['aa_mutations'],inplace=True)

In [122]:
gene_aa = {k:wuhan_seq[v.start:v.end].translate() for k,v in location_by_gene.items()}

In [124]:
gene_aa

{'ORF1a': Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...FLN'),
 'ORF1b': Seq('RVCGVSAARLTPCGTGTSTDVVYRAFDIYNDKVAGFAKFLKTNCCRFQEKDEDD...NN*'),
 'S': Seq('MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDL...YT*'),
 'S1': Seq('SQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFSNVTWFH...RAR'),
 'ORF3a': Seq('MDLFMRIFTIGTVTLKQGEIKDATPSDFVRATATIPIQASLPFGWLIVGVALLA...PL*'),
 'E': Seq('MYSFVSEETGTLIVNSVLLFLAFVVFLLVTLAILTALRLCAYCCNIVNVSLVKP...LV*'),
 'M': Seq('MADSNGTITVEELKKLLEQWNLVIGFLFLTWICLLQFAYANRNRFLYIIKLIFL...VQ*'),
 'ORF6': Seq('MFHLVDFQVTIAEILLIIMRTFKVSIWNLDYIINLIIKNLSKSLTENKYSQLDE...ID*'),
 'ORF7a': Seq('MKIILFLALITLATCELYHYQECVRGTTVLLKEPCSSGTYEGNSPFHPLADNKF...TE*'),
 'ORF7b': Seq('MIELSLIDFYLCFLAFLLFLVLIMLIIFWFSLELQDHNETCHA*'),
 'ORF8': Seq('MKFLVFLGIITTVAAFHQECSLQSCTQHQPYVVDDPCPIHFYSKWYIRVGARKS...FI*'),
 'N': Seq('MSDNGPQNQRNAPRITFGGPSDSTGSNQNGERSGARSKQRRPQGLPNNTASWFT...QA*'),
 'ORF9b': Seq('MDPKISEMHPALRLVDPQIQLAVTRMENAVGRDQNNVGPKVYPIILRLGSPLSL...VK*')}

In [133]:
def get_aa(row):
    return gene_aa[row['gene']][row['residue']-1]

In [17]:
## Classify mutation type
all_muts['residue'] = pd.to_numeric(all_muts.aa_mutation.str[1:-1])

# What type of mutation compared to Wuhan-Hu-1
#all_muts['new_aa'] = all_muts.aa_mutation.str[-1]
#all_muts['og_aa'] = all_muts['og_aa'] = all_muts.apply(get_aa,axis=1)
#all_muts['mut_type'] = np.where(all_muts.og_aa==all_muts.new_aa,'synonymous','missense')
#all_muts['mut_type'] = np.where(all_muts.aa_mutation.str[-1]=='*','nonsense',all_muts['mut_type'])


# What type of mutation compared to predecessor
all_muts['mut_type'] = np.where(all_muts.aa_mutation.str[-1]==all_muts.aa_mutation.str[0],'synonymous','missense')
all_muts['mut_type'] = np.where(all_muts.aa_mutation.str[-1]=='*','nonsense',all_muts['mut_type'])
all_muts['mut_type'] = np.where(all_muts.aa_mutation.str[0]=='*','undoStop',all_muts['mut_type'])

In [166]:
all_muts

,node_id,nt_mutations,codon_changes,leaves_sharing_mutations,gene,aa_mutation,mut_type,residue,og_aa,new_aa
0,CHN/YN-0306-466/2020|MT396241.1|2020-03-06,G15910T,GAT>TAT,1,ORF1b,D815Y,missense,815,D,Y
1,DP0803|LC571037.1|2020-02-17,G4162T,GTG>GTT,1,ORF1a,V1299V,synonymous,1299,V,V
2,node_2,T13090C,GAT>GAC,2,ORF1a,D4275D,synonymous,4275,D,D
3,England/LEED-2A8B52/2020|OA971832.1|2020-04-04,C1191T,CCA>CTA,1,ORF1a,P309L,missense,309,P,L
4,England/LEED-2A8B52/2020|OA971832.1|2020-04-04,C11674T,TAC>TAT,1,ORF1a,Y3803Y,synonymous,3803,Y,Y
...,...,...,...,...,...,...,...,...,...,...
5508953,Netherlands/NoordBrabant_60/2020|LR878078.1|20...,T6163A,GAT>GAA,1,ORF1a,D1966E,missense,1966,D,E
5508954,Netherlands/NoordBrabant_60/2020|LR878078.1|20...,C12923A,CCT>ACT,1,ORF1a,P4220T,missense,4220,P,T
5508955,Netherlands/NoordBrabant_60/2020|LR878078.1|20...,C13430T,CCC>TCC,1,ORF1a,P4389S,missense,4389,P,S
5508956,Netherlands/NoordBrabant_60/2020|LR878078.1|20...,C19366T,CCA>TCA,1,ORF1b,P1967S,missense,1967,P,S


In [167]:
gene_aa['S'][12:685]==gene_aa['S1']

True

In [18]:
 ## Add S1
S1 = all_muts[(all_muts.gene=='S')&(all_muts.residue >= 13)&(all_muts.residue<=685)].reset_index(drop=True)
S1['gene'] = 'S1'
final_muts = pd.concat([all_muts,S1])

In [169]:
S1.head()

,node_id,nt_mutations,codon_changes,leaves_sharing_mutations,gene,aa_mutation,mut_type,residue,og_aa,new_aa
0,England/PHEC-1E01E/2020|2020-04-03,C22445T,CCT>TCT,1,S1,P295S,missense,295,P,S
1,England/LIVE-A9C05/2020|2020-03-24,C23603T,CCT>TCT,1,S1,P681S,missense,681,P,S
2,node_28,G23611T,CGG>CGT,2,S1,R683R,synonymous,683,R,R
3,England/BIRM-61D60/2020|2020-03-25,C21767T,CAT>TAT,1,S1,H69Y,missense,69,H,Y
4,node_31,A23403G,GAT>GGT,2,S1,D614G,missense,614,D,G


In [19]:
final_muts.columns

Index(['node_id', 'nt_mutations', 'codon_changes', 'leaves_sharing_mutations',
       'gene', 'aa_mutation', 'residue', 'mut_type'],
      dtype='object')

In [20]:
len(final_muts)

5960462

In [54]:
from random import choices
from itertools import compress


In [103]:
%load_ext line_profiler

In [131]:
def flatten(l):
    return [item for sublist in l for item in sublist]

def draw_samples1(df,n):
    '''
    Draws random samples with replacement of node ids for bootstrap.
    Returns indexes of random samples.
    '''
    ids = df['node_id'].unique()
    #idxs = {ID:df.index[df['node_id']==ID].to_list() for ID in ids}
    idxs = {ID:list(np.where(df['node_id']==ID)) for ID in ids}
    length  = len(ids)
    sample_ids = {iterat:choices(ids,k=length) for iterat in range(n)}
    sample_idxs = {iterat:flatten([idxs[ID] for ID in sample_ids[iterat]]) for iterat in range(n)}
    return sample_idxs

def draw_samples2(df,n):
    '''
    Draws random samples with replacement of node ids for bootstrap.
    Returns indexes of random samples.
    '''
    ids = df['node_id'].unique()
    idxs = {ID:df.index[df['node_id']==ID].to_list() for ID in ids}
    #idxs = {ID:np.where(df['node_id']==ID) for ID in ids}
    length  = len(ids)
    sample_ids = {iterat:choices(ids,k=length) for iterat in range(n)}
    sample_idxs = {iterat:flatten([idxs[ID] for ID in sample_ids[iterat]]) for iterat in range(n)}
    return sample_idxs

def draw_samples(df,n):
    '''
    Draws random samples with replacement of node ids for bootstrap.
    Returns indexes of random samples.
    '''
    ids = df['node_id'].unique()
    idxs = {ID:np.flatnonzero(df['node_id']==ID) for ID in ids}
    length  = len(ids)
    sample_ids = {iterat:choices(ids,k=length) for iterat in range(n)}
    sample_idxs = {iterat:np.concatenate([idxs[ID] for ID in sample_ids[iterat]]) for iterat in range(n)}
    return sample_idxs


def draw_genes(samples,genes,df):
    '''
    Returns indexes of random samples for each gene.
    '''
    bootstrap_genes = {}
    for k in samples.keys():
        bootstrap_genes[k] = {}
        for gene in genes:
            filt = df.iloc[samples[k],4].values == gene
            subset = compress(samples[k],filt)
            bootstrap_genes[k][gene] = list(subset)
    return bootstrap_genes

def draw_counts(gene_samples,genes,df):
    '''
    Returns df of counts per gene by mutation type for each iteration.
    '''
    iterations = []
    the_genes = []
    mutTypes = []
    counts = []
    for k in gene_samples.keys():
        iterations.extend([k]*len(genes)*3)
        for gene in genes:
            the_genes.extend([gene]*3)
            for mut_type in ['synonymous','missense','nonsense']:
                filt = df.iloc[gene_samples[k][gene],7].values == mut_type
                subset = compress(gene_samples[k][gene],filt)
                count = sum(1 for x in subset)
                mutTypes.append(mut_type)
                counts.append(count)
    bootstrap = pd.DataFrame({'bootstrap':iterations,'gene':the_genes,'mutType':mutTypes,'count':counts})
    return bootstrap
        
## Next figure out how to draw muts
            


In [141]:
lprun -f draw_samples1 samples = draw_samples1(final_muts.iloc[0:1000,:],1000)

Timer unit: 1e-09 s

Total time: 54.7085 s
File: /var/folders/b5/2grxct1x69395r8j6vkk07bc0000gp/T/ipykernel_2572/538387251.py
Function: draw_samples1 at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def draw_samples1(df,n):
     5                                               '''
     6                                               Draws random samples with replacement of node ids for bootstrap.
     7                                               Returns indexes of random samples.
     8                                               '''
     9         1   28698000.0 28698000.0      0.1      ids = df['node_id'].unique()
    10                                               #idxs = {ID:df.index[df['node_id']==ID].to_list() for ID in ids}
    11         1 54550195000.0 54550195000.0     99.7      idxs = {ID:list(np.where(df['node_id']==ID)) for ID in ids}
    12         1       4000.0   4000.0      0.0      length  

In [142]:
lprun -f draw_samples2 samples = draw_samples2(final_muts.iloc[0:50000,:],10)

Timer unit: 1e-09 s

Total time: 55.1689 s
File: /var/folders/b5/2grxct1x69395r8j6vkk07bc0000gp/T/ipykernel_2572/538387251.py
Function: draw_samples2 at line 17

Line #      Hits         Time  Per Hit   % Time  Line Contents
    17                                           def draw_samples2(df,n):
    18                                               '''
    19                                               Draws random samples with replacement of node ids for bootstrap.
    20                                               Returns indexes of random samples.
    21                                               '''
    22         1    5291000.0 5291000.0      0.0      ids = df['node_id'].unique()
    23         1 55017066000.0 55017066000.0     99.7      idxs = {ID:df.index[df['node_id']==ID].to_list() for ID in ids}
    24                                               #idxs = {ID:np.where(df['node_id']==ID) for ID in ids}
    25         1       9000.0   9000.0      0.0      length  = len(

In [157]:
lprun -f draw_samples3 samples = draw_samples3(final_muts.iloc[0:1000,:],1000)

Timer unit: 1e-09 s

Total time: 0.488776 s
File: /var/folders/b5/2grxct1x69395r8j6vkk07bc0000gp/T/ipykernel_2572/538387251.py
Function: draw_samples3 at line 30

Line #      Hits         Time  Per Hit   % Time  Line Contents
    30                                           def draw_samples3(df,n):
    31                                               '''
    32                                               Draws random samples with replacement of node ids for bootstrap.
    33                                               Returns indexes of random samples.
    34                                               '''
    35         1    5630000.0 5630000.0      1.2      ids = df['node_id'].unique()
    36                                               #idxs = {ID:df.index[df['node_id']==ID].to_list() for ID in ids}
    37                                               #idxs = {ID:list(np.where(df['node_id']==ID)) for ID in ids}
    38         1  105645000.0 105645000.0     21.6      idxs = {I

In [158]:
gene_samples = draw_genes(samples,all_genes,final_muts)

In [159]:
counts = draw_counts(gene_samples,all_genes,final_muts)

In [160]:
counts

,bootstrap,gene,mutType,count
0,0,ORF1a,synonymous,183
1,0,ORF1a,missense,239
2,0,ORF1a,nonsense,0
3,0,ORF1b,synonymous,85
4,0,ORF1b,missense,126
...,...,...,...,...
38995,999,N,missense,68
38996,999,N,nonsense,0
38997,999,ORF9b,synonymous,2
38998,999,ORF9b,missense,16


In [166]:
def get_ci_counts(df,genes,ci):
    '''
    Given df of bootstrap counts, returns dictionary of min & max ci counts.
    '''
    min_counts = {}
    max_counts = {}
    for mutType in df['mutType'].unique():
        min_counts[mutType] = {}
        max_counts[mutType] = {}
        for gene in genes:
            values = df[(df.gene==gene) & (df.mutType==mutType)]['count']
            minimum = 100-ci/2
            maximum = 100-minimum
            min_counts[mutType][gene] = np.percentile(values,minimum)
            max_counts[mutType][gene] = np.percentile(values,maximum)
    return min_counts, max_counts

In [167]:
minimum_counts,maximum_counts = get_ci_counts(counts,all_genes,95)

In [168]:
minimum_counts

{'synonymous': {'ORF1a': 181.0,
  'ORF1b': 83.0,
  'S': 40.0,
  'S1': 0.0,
  'ORF3a': 11.0,
  'E': 6.0,
  'M': 12.0,
  'ORF6': 5.0,
  'ORF7a': 4.0,
  'ORF7b': 2.0,
  'ORF8': 6.0,
  'N': 28.0,
  'ORF9b': 3.0},
 'missense': {'ORF1a': 241.0,
  'ORF1b': 136.0,
  'S': 80.0,
  'S1': 0.0,
  'ORF3a': 37.0,
  'E': 9.0,
  'M': 11.0,
  'ORF6': 5.0,
  'ORF7a': 5.0,
  'ORF7b': 2.0,
  'ORF8': 12.0,
  'N': 61.0,
  'ORF9b': 15.0},
 'nonsense': {'ORF1a': 0.0,
  'ORF1b': 1.0,
  'S': 1.0,
  'S1': 0.0,
  'ORF3a': 1.0,
  'E': 0.0,
  'M': 0.0,
  'ORF6': 1.0,
  'ORF7a': 1.0,
  'ORF7b': 0.0,
  'ORF8': 3.0,
  'N': 0.0,
  'ORF9b': 0.0}}

In [53]:
grab = samples[0]

final_muts.iloc[grab,]._is_view

False

In [144]:
denominators = {'synonymous':syn_denominators,'missense':nonsyn_denominators,'nonsense':stop_denominators}


In [151]:
def get_counts(df, genes):
    all_syn = {gene:len(df[(df.gene==gene)&(df.mut_type=='synonymous')]) for gene in genes}
    all_missense = {gene:len(df[(df.gene==gene)&(df.mut_type=='missense')]) for gene in genes}
    all_nonsense = {gene:len(df[(df.gene==gene)&(df.mut_type=='nonsense')]) for gene in genes}
    counts = {'synonymous':all_syn,'missense':all_missense,'nonsense':all_nonsense}
    return counts

def get_differences(counts,denominators,genes):
    diffs = {}
    for mutType in counts.keys():
        diffs[mutType] = {gene:counts[mutType][gene]/denominators[mutType][gene] for gene in genes}
    return diffs

In [149]:
counted = get_counts(final_muts.iloc[0:1000],all_genes)

In [ ]:
counte

In [153]:
get_differences(counted,denominators,all_genes)

{'synonymous': {'ORF1a': 0.1864679388061965,
  'ORF1b': 0.14086567558329405,
  'S': 0.13997138285077795,
  'S1': 0.0,
  'ORF3a': 0.1697515146464685,
  'E': 0.3080999476230088,
  'M': 0.21937922990577594,
  'ORF6': 0.3968789439845057,
  'ORF7a': 0.14652175664934064,
  'ORF7b': 0.17931751752828728,
  'ORF8': 0.22262211750737437,
  'N': 0.2932222719071044,
  'ORF9b': 0.13157029142819554},
 'missense': {'ORF1a': 0.11834472945611134,
  'ORF1b': 0.1084064461203143,
  'S': 0.13273030093319402,
  'S1': 0.0,
  'ORF3a': 0.28080507574147284,
  'E': 0.24323666927920887,
  'M': 0.1039266315772553,
  'ORF6': 0.18016719515710647,
  'ORF7a': 0.08403431961613078,
  'ORF7b': 0.09505884142284086,
  'ORF8': 0.21049418772924017,
  'N': 0.31832774607256925,
  'ORF9b': 0.3200805749500666},
 'nonsense': {'ORF1a': 0.0,
  'ORF1b': 0.013531140566900574,
  'S': 0.025303771780221574,
  'S1': 0.0,
  'ORF3a': 0.1252960118279438,
  'E': 0.0,
  'M': 0.0,
  'ORF6': 0.4948780125699015,
  'ORF7a': 0.2916132042458882,
  '

## 3. Calculate dS - dN - dStop

In [170]:
all_syn = {gene:len(final_muts[(final_muts.gene==gene)&(final_muts.mut_type=='synonymous')]) for gene in all_genes}
all_missense = {gene:len(final_muts[(final_muts.gene==gene)&(final_muts.mut_type=='missense')]) for gene in all_genes}
all_nonsense = {gene:len(final_muts[(final_muts.gene==gene)&(final_muts.mut_type=='nonsense')]) for gene in all_genes}

In [176]:
all_syn

{'ORF1a': 1006032,
 'ORF1b': 530552,
 'S': 273119,
 'S1': 153726,
 'ORF3a': 76980,
 'E': 14541,
 'M': 80598,
 'ORF6': 19985,
 'ORF7a': 34325,
 'ORF7b': 8369,
 'ORF8': 23949,
 'N': 152729,
 'ORF9b': 20238}

In [171]:
dS = {gene:all_syn[gene]/syn_denominators[gene] for gene in all_genes}
dN = {gene:all_missense[gene]/nonsyn_denominators[gene] for gene in all_genes}
dStop = {gene:all_nonsense[gene]/stop_denominators[gene] for gene in all_genes}

In [172]:
to_print = []
for gene in all_genes:
    to_print.append({'gene':gene, 'dS': dS[gene],'dN': dN[gene],  'dStop': dStop[gene], 'dN_dS':dN[gene]/dS[gene], 'dStop_dS':dStop[gene]/dS[gene]})
    
df_to_print = pd.DataFrame(to_print)
print(df_to_print)
df_to_print.to_csv('usher/trimmed/dn_ds_dstop_usher.tsv')

     gene           dS           dN        dStop     dN_dS  dStop_dS
0   ORF1a  1042.181741   652.548910     7.151088  0.626137  0.006862
1   ORF1b   900.440553   475.495566     5.656017  0.528070  0.006281
2       S   955.721103   804.036480     9.286484  0.841288  0.009717
3      S1  1000.057899   940.513171    12.685169  0.940459  0.012684
4   ORF3a  1187.951963  1491.401293   210.998484  1.255439  0.177615
5       E   746.680223   648.063566    14.236142  0.867927  0.019066
6       M  1473.460598   602.698880    22.651685  0.409036  0.015373
7    ORF6  1586.325139  1006.990487   826.446281  0.634795  0.520982
8   ORF7a  1257.339824  1849.175203  2533.535518  1.470704  2.014997
9   ORF7b   750.354152  1059.668435  1429.672117  1.412224  1.905330
10   ORF8   888.596182  1495.683992  2170.407141  1.683199  2.442512
11      N  1599.412299  1269.877219    15.891795  0.793965  0.009936
12  ORF9b   887.573186  1352.063026   494.339485  1.523326  0.556956


In [ ]:
## I'm really struggling to understand why I'm getting such different values for S & S1 compared to Katie.

## 4. Calculate dN - dS - dStop, excluding terminal branches

In [173]:
noTerm_syn = {gene:len(final_muts[(final_muts.gene==gene)&(final_muts.mut_type=='synonymous')&(final_muts.node_id.str.contains('node'))]) for gene in all_genes}
noTerm_missense = {gene:len(final_muts[(final_muts.gene==gene)&(final_muts.mut_type=='missense')&(final_muts.node_id.str.contains('node'))]) for gene in all_genes}
noTerm_nonsense = {gene:len(final_muts[(final_muts.gene==gene)&(final_muts.mut_type=='nonsense')&(final_muts.node_id.str.contains('node'))]) for gene in all_genes}

In [174]:
dS_noTerm = {gene:noTerm_syn[gene]/syn_denominators[gene] for gene in all_genes}
dN_noTerm = {gene:noTerm_missense[gene]/nonsyn_denominators[gene] for gene in all_genes}
dStop_noTerm = {gene:noTerm_nonsense[gene]/stop_denominators[gene] for gene in all_genes}

In [175]:
to_print_noTerm = []
for gene in all_genes:
    to_print_noTerm.append({'gene':gene, 'dS': dS_noTerm[gene],'dN': dN_noTerm[gene],  'dStop': dStop_noTerm[gene], 'dN_dS':dN_noTerm[gene]/dS_noTerm[gene], 'dStop_dS':dStop_noTerm[gene]/dS_noTerm[gene]})
    
df_to_print_noTerm = pd.DataFrame(to_print_noTerm)
print(df_to_print_noTerm)
df_to_print_noTerm.to_csv('usher/trimmed/dn_ds_dstop_usher_noTerminalBranches.tsv')

     gene          dS          dN       dStop     dN_dS  dStop_dS
0   ORF1a  399.959226  236.201348    0.479448  0.590564  0.001199
1   ORF1b  348.023078  168.751095    0.257092  0.484885  0.000739
2       S  371.015146  267.927033    0.860328  0.722146  0.002319
3      S1  388.057022  307.709134    1.326901  0.792948  0.003419
4   ORF3a  464.995694  572.933426   55.004949  1.232126  0.118291
5       E  274.619753  205.048512    4.448794  0.746663  0.016200
6       M  566.985620  184.677624    3.415730  0.325718  0.006024
7    ORF6  605.081638  344.695878  247.439006  0.569668  0.408935
8   ORF7a  489.236145  652.392037  894.960924  1.333491  1.829303
9   ORF7b  270.052181  378.761954  556.459207  1.402551  2.060562
10   ORF8  339.684248  575.543733  891.887483  1.694349  2.625637
11      N  621.788300  437.345794    2.707120  0.703368  0.004354
12  ORF9b  314.102142  491.515731  105.201849  1.564828  0.334929


In [ ]:
## Think about bootstrapping these values??

## ORF7a dN/dS is much lower when comparing to OG sequence --> suggests high rate of back mutation

## Actually katie did not compare to OG -- she compared to predecessor

## 5. Just by gene length

In [183]:
lengths = {k:(v.end - v.start)/3 for k,v in location_by_gene.items()}

In [184]:
syn_scaled = {gene:all_syn[gene]/lengths[gene] for gene in all_genes}
missense_scaled = {gene:all_missense[gene]/lengths[gene] for gene in all_genes}
stop_scaled = {gene:all_nonsense[gene]/lengths[gene] for gene in all_genes}

In [186]:
to_print_scaled = []
for gene in all_genes:
    to_print_scaled.append({'gene':gene, 'syn': syn_scaled[gene],'missense':missense_scaled[gene], 'stop': stop_scaled[gene]})
    
df_to_print_scaled = pd.DataFrame(to_print_scaled)
print(df_to_print_scaled)
#df_to_print_noTerm.to_csv('usher/trimmed/dn_ds_dstop_usher_noTerminalBranches.tsv')

     gene         syn    missense       stop
0   ORF1a  228.591684  301.946376   0.199955
1   ORF1b  196.792285  219.636869   0.155045
2       S  214.379121  375.632653   0.288069
3      S1  228.419019  442.098068   0.355126
4   ORF3a  278.913043  712.003623   6.101449
5       E  191.328947  315.513158   0.210526
6       M  361.426009  286.062780   0.565022
7    ORF6  322.338710  450.741935  26.935484
8   ORF7a  281.352459  901.844262  71.213115
9   ORF7b  190.204545  506.704545  37.954545
10   ORF8  196.303279  698.909836  76.336066
11      N  363.640476  579.385714   0.754762
12  ORF9b  206.510204  646.551020  13.857143
